In [ ]:
# установка Spark в Colab

!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 199 kB 54.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=55bea0aa33f3909708c66cb92d27fe5c0db22da42f0978ab3e243707f8630d59
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
# импорт необходимых библиотек

from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

import pyspark.sql.types as T
import pyspark.sql.functions as F

from datetime import datetime

In [4]:
conf = SparkConf()
conf.setMaster('local').setAppName('task3_3')

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)


print('Spark: ', spark.version)

Spark:  3.3.1


a. Создать схему фрейма данных, которая включает следующие атрибуты: 

*   id - уникальный идентификатор посетителя сайта. Тип – последовательность чисел фиксированной длины. Данное поле не является первичным ключом.
*   timestamp – дата и время события в формате unix timestamp.
*   type – тип события, значение из списка (факт посещения(visit), клик по визуальному элементу страницы(click), скролл(scroll), перед на другую страницу(move)).
*   page_id – id текущей страницы. Тип - последовательность чисел фиксированной длины.
*   tag – каждая страница с новостью размечается редакцией специальными тегами, которые отражают тематику конкретной новости со страницы. Возможный список тематик: политика, спорт, медицина и т.д.
*   sign – наличие у пользователя личного кабинета. Значения – True/False.

In [5]:
schema = T.StructType([
                       T.StructField('id', T.IntegerType(), False),
                       T.StructField('timestamp', T.LongType(), True),
                       T.StructField('type', T.StringType(), True),
                       T.StructField('page_id', T.IntegerType(), True),
                       T.StructField('tag', T.StringType(), True),
                       T.StructField('sign', T.BooleanType(), True),
                       ])

b. Создать датафрейм с описанной выше схемой данных.

c. Наполнить датафрейм данными. 

Пример: (12345, 1667627426, 'click', 101, 'Sport', False)

In [30]:
webData = [(101, 1667202347, 'visit', 101, 'Sport', False),
           (101, 1667202565, 'move', 101, 'Sport', False),
           (101, 1667202347, 'click', 101, 'Sport', False),
           (101, 1667803547, 'visit', 222, 'Politics', False),
           (101, 1638941987, 'click', 202, 'Politics', False),
           (101, 1619160500, 'move', 311, 'Politics', False),           
           (21, 1634943260, 'visit', 412, 'Science', True),
           (21, 1626103180, 'scroll', 140, 'Medicine', True),
           (21, 1618119021, 'click', 431, 'Science', True),
           (21, 1626071935, 'visit', 552, 'Business', True),
           (21, 1620385526, 'click', 502, 'Medicine', True),
           (21, 1636887856, 'visit', 611, 'Medicine', True),
           (21, 1612165677, 'move', 345, 'Science', True),           
           (303, 1626960036, 'visit', 101, 'Sport', False),
           (303, 1626969999, 'click', 101, 'Sport', False),
           (303, 1638007179, 'scroll', 101, 'Sport', False),
           (303, 1617209174, 'visit', 107, 'Sport', False),
           (303, 1629474919, 'scroll', 104, 'Sport', False),
           (303, 1626960036, 'visit', 250, 'Sport', False),
           (303, 1629863841, 'scroll', 250, 'Sport', False),           
           (45, 1611557147, 'visit', 151, 'Fashion', False),
           (45, 1611560747, 'visit', 151, 'Fashion', False),
           (45, 1611564347, 'visit', 151, 'Fashion', False),           
           (115, 1631865594, 'move', 101, 'Sport', True),
           (115, 1636070942, 'move', 101, 'Sport', True),
           (115, 1640307636, 'visit', 502, 'Medicine', True),
           (115, 1618866891, 'click', 311, 'Politics', True),
           (121, 1626960036, 'visit', 151, 'Fashion', True),
           (121, 1638007179, 'scroll', 151, 'Fashion', True),
           (121, 1617209174, 'visit', 708, 'Science', True),
           (121, 1629474919, 'scroll', 316, 'Science', True),
           (121, 1626960036, 'click', 406, 'Science', True),
           (222, 1626960036, 'visit', 131, 'Politics', False),
           (222, 1638007179, 'scroll', 154, 'Politics', False),
           (222, 1617209174, 'visit', 234, 'Business', False),
           (222, 1629474919, 'scroll', 505, 'Business', False),
           (222, 1626960036, 'visit', 201, 'Business', False),
          ]

In [31]:
df = spark.createDataFrame(data=webData, schema=schema)

df = df.select(*[i for i in df.columns if i != 'timestamp'],
          F.from_unixtime('timestamp').alias('event_time'))

df.sort('id').show()

+---+------+-------+--------+-----+-------------------+
| id|  type|page_id|     tag| sign|         event_time|
+---+------+-------+--------+-----+-------------------+
| 21| visit|    611|Medicine| true|2021-11-14 11:04:16|
| 21| click|    431| Science| true|2021-04-11 05:30:21|
| 21|  move|    345| Science| true|2021-02-01 07:47:57|
| 21| visit|    552|Business| true|2021-07-12 06:38:55|
| 21| visit|    412| Science| true|2021-10-22 22:54:20|
| 21|scroll|    140|Medicine| true|2021-07-12 15:19:40|
| 21| click|    502|Medicine| true|2021-05-07 11:05:26|
| 45| visit|    151| Fashion|false|2021-01-25 06:45:47|
| 45| visit|    151| Fashion|false|2021-01-25 08:45:47|
| 45| visit|    151| Fashion|false|2021-01-25 07:45:47|
|101|  move|    101|   Sport|false|2022-10-31 07:49:25|
|101| click|    101|   Sport|false|2022-10-31 07:45:47|
|101| visit|    222|Politics|false|2022-11-07 06:45:47|
|101| visit|    101|   Sport|false|2022-10-31 07:45:47|
|101| click|    202|Politics|false|2021-12-08 05

d. Решить следующие задачи:

*   Вывести топ-5 самых активных посетителей сайта.
*   Посчитать процент посетителей, у которых есть ЛК.
*   Вывести топ-5 страниц сайта по показателю общего кол-ва кликов на данной странице.
*   Добавьте столбец к фрейму данных со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.).
*   Выведите временной промежуток на основе предыдущего задания, в течение которого было больше всего активностей на сайте.
*   Создайте второй фрейм данных, который будет содержать информацию о ЛК посетителя сайта со следующим списком атрибутов:

1.   Id – уникальный идентификатор личного кабинета
2.   User_id – уникальный идентификатор посетителя
3.   ФИО посетителя
4.   Дату рождения посетителя
5.   Дата создания ЛК

*   Вывести фамилии посетителей, которые читали хотя бы одну новость про спорт.


In [32]:
# топ-5 самых активных посетителей сайта

df.groupby('id').count().orderBy('count', ascending=False).show(5)

+---+-----+
| id|count|
+---+-----+
|303|    7|
| 21|    7|
|101|    6|
|121|    5|
|222|    5|
+---+-----+
only showing top 5 rows



In [35]:
# процент посетителей, у которых есть ЛК

usersAll = df.groupBy('id').count().distinct().count()
usersRegisted = df.filter(df['sign']=='true').groupBy('id', 'sign').count().distinct().count()
percent = (100*usersRegisted/usersAll)

print('Процент посетителей, у которых есть ЛК составляет: ', percent, '%')

Процент посетителей, у которых есть ЛК составляет:  42.857142857142854 %


In [37]:
# топ-5 страниц сайта по показателю общего кол-ва кликов на данной странице

df.filter(df['type']=='click').groupBy('page_id').count().orderBy('count', ascending=False).show(5)

+-------+-----+
|page_id|count|
+-------+-----+
|    101|    2|
|    406|    1|
|    502|    1|
|    431|    1|
|    202|    1|
+-------+-----+
only showing top 5 rows



In [39]:
# добавление столбца к фрейму данных со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.)

timeDF = df.withColumn('time_range', F.floor(F.hour('event_time') / F.lit(4)))
timeDF.show()

+---+------+-------+--------+-----+-------------------+----------+
| id|  type|page_id|     tag| sign|         event_time|time_range|
+---+------+-------+--------+-----+-------------------+----------+
|101| visit|    101|   Sport|false|2022-10-31 07:45:47|         1|
|101|  move|    101|   Sport|false|2022-10-31 07:49:25|         1|
|101| click|    101|   Sport|false|2022-10-31 07:45:47|         1|
|101| visit|    222|Politics|false|2022-11-07 06:45:47|         1|
|101| click|    202|Politics|false|2021-12-08 05:39:47|         1|
|101|  move|    311|Politics|false|2021-04-23 06:48:20|         1|
| 21| visit|    412| Science| true|2021-10-22 22:54:20|         5|
| 21|scroll|    140|Medicine| true|2021-07-12 15:19:40|         3|
| 21| click|    431| Science| true|2021-04-11 05:30:21|         1|
| 21| visit|    552|Business| true|2021-07-12 06:38:55|         1|
| 21| click|    502|Medicine| true|2021-05-07 11:05:26|         2|
| 21| visit|    611|Medicine| true|2021-11-14 11:04:16|       

In [40]:
# временной промежуток на основе предыдущего задания, в течение которого было больше всего активностей на сайте

maxTimeDF = timeDF.groupBy('time_range').count().orderBy('count', ascending = False)
dictRange = {
              0 : '0-4',
              1 : '4-8',
              2 : '8-12',
              3 : '12-16',
              4 : '16-20',
              5 : '20-0'}

In [60]:
maxTimeDF.select(F.when(maxTimeDF['time_range'].cast('int') == 0, dictRange[0]).when(maxTimeDF['time_range'].cast('int') == 1, dictRange[1]).when(maxTimeDF['time_range'].cast('int') == 2, dictRange[2]).when(maxTimeDF['time_range'].cast('int') == 3, dictRange[3]).when(maxTimeDF['time_range'].cast('int') == 4, dictRange[4]).otherwise(dictRange[5]).alias('period'), maxTimeDF['count']).show(1)

+------+-----+
|period|count|
+------+-----+
|   4-8|   12|
+------+-----+
only showing top 1 row



In [61]:
# создание второго фрейма данных, который содержит информацию о ЛК посетителя сайта

schemaUserLK = T.StructType([
                              T.StructField('id', T.IntegerType(), True),
                              T.StructField('user_id', T.IntegerType(), True),
                              T.StructField('fio', T.StringType(), True),
                              T.StructField('date_bd', T.DateType(), True),
                              T.StructField('date_create', T.DateType(), True),
                            ])

# заполнение
dataLK = [
            (101, 21, 'Жигалов Артур Александрович', datetime.strptime('1994-12-31', '%Y-%m-%d'), datetime.strptime('2019-08-29', '%Y-%m-%d')),
            (102, 115, 'Давыдов Антон Борисович', datetime.strptime('1983-01-29', '%Y-%m-%d'), datetime.strptime('2019-03-25', '%Y-%m-%d')),
            (102, 121, 'Крюкова Кира Васильевна', datetime.strptime('1997-02-05', '%Y-%m-%d'), datetime.strptime('2019-07-11', '%Y-%m-%d'))
         ]

lkDF = spark.createDataFrame(data=dataLK, schema=schemaUserLK)

In [65]:
# фамилии посетителей, которые читали хотя бы одну новость про спорт

df.join(lkDF, df['id'] == lkDF['user_id']).where(df['tag']=='Sport').groupBy(df['id'], 'fio').count().orderBy(df['id']).show()

+---+--------------------+-----+
| id|                 fio|count|
+---+--------------------+-----+
|115|Давыдов Антон Бор...|    2|
+---+--------------------+-----+

